In [14]:
import os
import wmill
import requests 
import pandas as pd 
import mysql.connector 
import decimal 
import pymysql 
import psycopg2
 
url = "https://churchofjesuschristtemples.org/maps/gps/" 
filename = "temples.csv" 
 
# Send a GET request to the URL 
response = requests.get(url) 
 
# Check if the request was successful 
if response.status_code == 200: 
    # Save the content of the response (CSV file) to a local file 
    with open(filename, "wb") as file: 
        file.write(response.content)  
else: 
    print("Failed to download the file. Status code:", response.status_code)

temples = pd.read_csv(filename, encoding='latin1', usecols= ['Temple','Status','Latitude','Longitude','Address','City','County','State','Postal Code','Country','Phone'])

print(temples)

                              Temple              Status   Latitude  \
0                 Aba Nigeria Temple           Dedicated   5.147644   
1         Abidjan Ivory Coast Temple  Under Construction   5.362307   
2                 Accra Ghana Temple           Dedicated   5.567490   
3          Adelaide Australia Temple           Dedicated -34.892473   
4         Alabang Philippines Temple  Under Construction  14.412913   
..                               ...                 ...        ...   
300       Winchester Virginia Temple           Announced  39.188273   
301         Winnipeg Manitoba Temple           Dedicated  49.800544   
302  Winter Quarters Nebraska Temple           Dedicated  41.334075   
303                 Yigo Guam Temple           Dedicated  13.541741   
304    Yorba Linda California Temple  Under Construction  33.898375   

      Longitude                                            Address  \
0      7.356719                               72-80 Okpu-Umuobo Rd   
1     -

In [15]:
# Format my dataframe to be uploaded to database
column_names = {'Temple': 'Temple'
    , 'Status': 'Status'
    , 'Latitude': 'Latitude'
    , 'Longitude': 'Longitude'
    , 'Address': 'Address'
    , 'City': 'City'
    , 'County': 'County'
    , 'State': 'State'
    , 'Postal_Code': 'Postal Code'
    , 'Country': 'Country'
    , 'Phone': 'Phone'
}
df = temples.rename(columns=column_names)
df.head()

,Temple,Status,Latitude,Longitude,Address,City,County,State,Postal Code,Country,Phone
0,Aba Nigeria Temple,Dedicated,5.147644,7.356719,72-80 Okpu-Umuobo Rd,Aba,NaN,Abia State,NaN,Nigeria,(+234) 80-3903-4810
1,Abidjan Ivory Coast Temple,Under Construction,5.362307,-3.976865,1874 Boulevard de France,Abidjan,NaN,NaN,NaN,Cote d'Ivoire,NaN
2,Accra Ghana Temple,Dedicated,5.567490,-0.193706,57 Independence Ave North Ridge P.M.B. CT 209 ...,Accra,NaN,NaN,NaN,Ghana,(+233) 302-650-123
3,Adelaide Australia Temple,Dedicated,-34.892473,138.635002,53-59 Lower Portrush Rd,Marden,NaN,South Australia,5070,Australia,(+61) 8-8363-8000
4,Alabang Philippines Temple,Under Construction,14.412913,121.037838,Filinvest Avenue and Corporate Avenue Alabang,Muntinlupa,NaN,Metro Manila,NaN,Philippines,NaN


In [16]:
 # Connect to Postgresql server
cnx = psycopg2.connect(
    host = wmill.get_variable("u/jckober5/remote_host")
    , port = 5432
    , user = wmill.get_variable("u/jckober5/remote_user")
    , password = wmill.get_variable("u/jckober5/remote_password")
    , database = 'data_lake'
)

# Create a cursor object to execute SQL queries\n",
cursor = cnx.cursor()

# Query to drop table\n",
drop_table_query = "drop table lds_temples.temples"
cursor.execute(drop_table_query)
# Query to create table
create_table_query = "create table lds_temples.temples (Temple varchar(200), Status varchar(50), Latitude varchar(50), Longitude varchar(50), Address varchar(200), City varchar(50), County varchar(50), State varchar(50), Postal_Code varchar(50), Country varchar(50), Phone varchar(50))"
cursor.execute(create_table_query)

In [17]:
# Insert Data into the table
for i in range(len(df)):
    # SQL query without parentheses
    sql = "INSERT INTO lds_temples.temples (Temple, Status, Latitude, Longitude, Address, City, County, State, Postal_Code, Country, Phone) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
    
    # Values as a list using .tolist()
    values = (
        str(df['Temple'].iloc[i])[:50],
        str(df['Status'].iloc[i])[:50],
        str(df['Latitude'].iloc[i])[:50],
        str(df['Longitude'].iloc[i])[:50],
        str(df['Address'].iloc[i])[:50],
        str(df['City'].iloc[i])[:50],
        str(df['County'].iloc[i])[:50],
        str(df['State'].iloc[i])[:50],
        str(df['Postal Code'].iloc[i])[:50],
        str(df['Country'].iloc[i])[:50],
        str(df['Phone'].iloc[i])[:50]
    )

    # Execute the INSERT statement
    cursor.execute(sql, values)
# Commit the changes and close the connection\n",
cnx.commit()
cnx.close()

In [18]:
url = "https://churchofjesuschristtemples.org/statistics/locations/" 
filename = "UnitsPerCountry.csv" 
 
# Send a GET request to the URL 
response = requests.get(url) 
 
# Check if the request was successful 
if response.status_code == 200: 
    # Save the content of the response (CSV file) to a local file 
    with open(filename, "wb") as file: 
        file.write(response.content) 
else: 
    print("Failed to download the file. Status code:", response.status_code)